In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
(Dataset, _),(_, _) = tf.keras.datasets.fashion_mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
Dataset = Dataset / 127.5 - 1.0
Dataset = np.expand_dims(Dataset, axis=3)

In [ ]:
#Discriminator
Discriminator = tf.keras.Sequential(
    [
      tf.keras.Input(shape=(28,28,1)),
      tf.keras.layers.Conv2D(64,kernel_size=5,strides=2,padding='same'),  #output dimensions 14,14,64
      tf.keras.layers.LeakyReLU(0.2),
      tf.keras.layers.Dropout(0.3),
     
      tf.keras.layers.Conv2D(128,kernel_size=5,strides=2,padding='same'), #output dimensions 7,7,128
      tf.keras.layers.LeakyReLU(0.2),
      tf.keras.layers.Dropout(0.3),
     
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(1,activation='sigmoid')
    ]
)

latent_dim = 100  #dimension of noise vector

#Genrator
Generator = tf.keras.Sequential(
    [
     tf.keras.Input(shape=(latent_dim)),
     tf.keras.layers.Dense(7*7*256),
     tf.keras.layers.LeakyReLU(0.2),
     tf.keras.layers.BatchNormalization(),
     tf.keras.layers.Reshape(target_shape=(7,7,256)),                             #output dimensions 7,7,256

     tf.keras.layers.Conv2DTranspose(128,kernel_size=5,strides=1,padding='same'), #output dimensions 7,7,128
     tf.keras.layers.LeakyReLU(0.2),
     tf.keras.layers.BatchNormalization(),

     tf.keras.layers.Conv2DTranspose(64,kernel_size=5,strides=2,padding='same'),  #output dimensions 14,14,64
     tf.keras.layers.LeakyReLU(0.2),
     tf.keras.layers.BatchNormalization(),

     tf.keras.layers.Conv2DTranspose(1,kernel_size=5,strides=2,padding='same',activation='tanh') #output dimensions 28,28,1 which is our image's dimensions
    ]
)

opt_disc = tf.keras.optimizers.Adam(1e-4)
opt_gen = tf.keras.optimizers.Adam(1e-4)
epoch = 50
batchsize = 32
loss_func = tf.keras.losses.BinaryCrossentropy()

In [ ]:
#training loop

for epoch in range(10):
  for index, real_image in enumerate(Dataset):
    random_latent_vectors = tf.random.normal(shape = (batchsize,latent_dim))
    gen_fake = Generator(random_latent_vectors)
    actual_image = Dataset[np.random.randint(0,Dataset.shape[0],batchsize)]
    print("index = ",index)
    if index%100 == 0:
      img = tf.keras.preprocessing.image.array_to_img(gen_fake[0])
      img.save(f'/content/drive/MyDrive/MNIST/image{index}.png')

    #training of discriminator
    with tf.GradientTape() as disc_tape:
      fake_loss = loss_func(tf.zeros(shape=(batchsize,1)),Discriminator(gen_fake))
      real_loss = loss_func(tf.ones(shape=(batchsize,1)),Discriminator(actual_image))
      total_disc_loss = (fake_loss + real_loss)/2
    grades = disc_tape.gradient(total_disc_loss,Discriminator.trainable_weights)
    opt_disc.apply_gradients(zip(grades,Discriminator.trainable_weights))
    # print(len(grades))
    print(total_disc_loss)
    #training of Generator 
    with tf.GradientTape() as gen_tape:
      gen_fake = Generator(random_latent_vectors)
      output = Discriminator(gen_fake)
      total_gen_loss = loss_func(tf.ones(shape=(batchsize,1)),output)
    grades = gen_tape.gradient(total_gen_loss,Generator.trainable_weights)
    # print(grades)
    # print(grades)
    print(total_gen_loss)
    opt_gen.apply_gradients(zip(grades,Generator.trainable_weights))


